<center>
<center><h1>PROGRAMMING ASSIGNMENT 4</h1></center>

<img align="center" width="60%" src="https://raw.githubusercontent.com/fahadsultan/csc343/main/programming_assignments/PA.4./ham_or_spam.png">

<h1>SPAM FILTER</h1>
</center>

You are given a file `emails.csv` containing Bag Of Words (BOW) representation of 5172 emails. 

For each email, there are 3000 comma-separated columns, each representing a word. 

For each email, you are also given an additional column **`Prediction`** indicating if the email is in fact **Ham** (0) or **Spam** (1).

The BOW representation of this data therefore is a $5172 \times 3001$ dimensional matrix $BOW$ where rows represent emails and columns represent words. 

The value of a cell in this matrix $BOW[i, j]=1$ if email $i$ contains the word $j$. Otherwise,  $BOW[i, j]=0$


In [ ]:
import pandas as pd

data = pd.read_csv("https://raw.githubusercontent.com/fahadsultan/csc343/main/programming_assignments/PA.4./emails_binary.csv", index_col=0)
data.head()

,the,to,ect,and,for,of,a,you,hou,in,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
Email No.,,,,,,,,,,,,,,,,,,,,,
Email 1,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Email 2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,0,0
Email 3,0,0,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
Email 4,0,1,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
Email 5,1,1,1,1,1,1,1,0,1,1,...,0,0,0,0,0,0,0,1,0,0



<br/>
<hr/>
<br/>

You are to implement a **Naive Bayes classifier** that <u>given an email predicts if it is Spam or not</u> using the following equation from class 

<br/>

$$ P(\text{Spam} | word_1, word_2, ... word_n )=  𝛂~P(\text{Spam})   \prod_{i}^{n}  P( word_i | \text{Spam}) 
   ~~~~~~\text{equation 1}$$ 

<br/>

Note that an email (or any document really) is just a collection of words i.e. $email = (word_1, word_2, ..., word_n)$

Any given email is classified as Spam if

<br/>

$$P(\text{spam} | ~ word_1, ~word_2, ... word_n ) > P(~\lnot ~ \text{spam} | ~word_1, ~word_2, ... ~ word_n )~~\text{equation 2}$$

<br/>

Since we are interested only in relative probabilities, we can ignore the normalization constant $ \mathbf{\alpha}$ in equaton 1. 

<br/>
<hr/>
<br/>

Note that contrast in capitalization of Spam in equations 1 and 2. This is because $P(\text{spam})$ is shorthand for $P(\text{Spam}=$True$)$ and $P(~\lnot ~ \text{spam})$ is shorthand for $P(\text{Spam}=$False$)$. 

On the other hand, $P(\text{Spam})$ is a probability distribution table: 
<center>

| Spam      | $P($Spam$)$ |
| ----------- | ----------- |
| True      | $p_1$       |
| False   | $p_2$        |

</center>

such that  $p_1 \in [0, 1], p_2 \in [0, 1]$ and $p_1 + p_2 = 1$. 

<br/>

<hr/>

<br/>

# Step 1. 


Speaking of, start by calculating  $P($Spam$)$ for use in equation 1.

* $P(spam) = \frac{  count(spam~emails)  }{ count(all~emails) }$

* $P(\lnot spam) = \frac{  count(ham~emails)  }{ count(all~emails) }$ or simply $1- P(spam)$


In [ ]:
from functools import total_ordering
#Write your code here for P(Spam)

def probSpam():
  spamCount = data['Prediction'].sum()
  totalEmails = len(data)

  pSpam = spamCount / totalEmails
  NotSpam = 1 - pSpam

  return (pSpam, NotSpam, totalEmails, spamCount, totalEmails - spamCount)

# print(PSpam * 100)
# print(NotSpam * 100)


<br/>
<hr/>

# Step 2. 
<br/>

For $P( word_i~|~\text{Spam})$ in equation 1, you would need to create the **join probability distribution** $P( word_{i}, \text{Spam})$ i.e. 

<center>

|  $\text{word}_{~i}$     | $\text{Spam}$ |$P(\text{word}_{~i}, \text{Spam})$ |
| ----------- | --| ----------- |
| True      |    True    | $jp_1 $
| False   | True        | $jp_2$
| True      | False       |$jp_3$
| False   | False       | $jp_4$

</center>
<br/>

where <br/>

$ jp_1= \frac{\text{count(spam emails containing}  ~word_i)} {\text{count(all emails})} $

<br/>

$ jp_2= \frac{\text{count(spam emails NOT containing}  ~word_i)} {\text{count(all emails})} $

<br/>

$ jp_3= \frac{\text{count(ham emails containing}  ~word_i)} {\text{count(all emails})} $

<br/>

$ jp_4= \frac{\text{count(ham emails NOT containing}  ~word_i)} {\text{count(all emails})} $

<br/>

 $jp_1, jp_2, jp_3, jp_4 \in [0, 1]$ and <br/>$jp_1 + jp_2 + jp_3 + jp_4 = 1$



In [ ]:

_tuple = [tuple(x) for x in data.values]
tuple_dict = {}
for i in range(len(_tuple)):
  tuple_dict.update({i: _tuple[i]})


In [ ]:
#Write your code here for P(word_i, Spam)
  #Spam is a list of possible values: True, False
from tqdm import tqdm

isSpam, notSpam, totalEmails, spamCount, hamCount = probSpam()

words = list(data.columns[:-1])

jointP_dict = {}
for i in tqdm(range(len(words))):
  inSpamEmail = 0
  inHamEmail = 0

  for t in tuple_dict.values():
    if t[i] == 1 and t[-1] == 1: # word in spam
      inSpamEmail += 1
    elif t[1] == 1 and t[-1] == 0: # word in ham
      inHamEmail += 1

  jp1 = inSpamEmail / totalEmails
  jp2 = (spamCount - inSpamEmail) / totalEmails
  jp3 = inHamEmail / totalEmails
  jp4 = (hamCount - inHamEmail) / totalEmails

  newTup = (jp1, jp2, jp3, jp4)
  jointP_dict.update({words[i]: newTup})
  

100%|██████████| 3000/3000 [00:35<00:00, 85.56it/s]


<br/>
<hr/>
<br/>

# Step 3. 

Once you have the joint probability distribution table above, you can compute the **conditional probability distribution table** using the formula: 
<br/>

<br/>
$$P( word_{~i} |~\text{Spam}) = P(\text{word}_{~i}, \text{Spam}) \cdot P(\text{Spam})$$


<br/>

That is: 

<center>

|  $\text{word}_{~i}$     | $\text{Spam}$ |$P(\text{word}_{~i} | \text{Spam})$ |
| ----------- | --| ----------- |
| True      |    True    | $cp_1$
| False   | True        | $cp_2$
| True      | False       |$cp_3$
| False   | False       | $cp_4$

</center>

where $cp_1, cp_2, cp_3, cp_4 \in [0, 1]$ and <br/>$cp_1 + cp_2 + cp_3 + cp_4 = 1$



In [ ]:
#Write your code here for P(word_i | Spam)
condP_dict = {}
isSpam, notSpam, totalEmails, spamCount, hamCount = probSpam()

for key, value in tqdm(jointP_dict.items()):
  cp1 = value[0] / isSpam
  cp2 = value[1] / isSpam
  cp3 = value[2] / notSpam
  cp4 = value[3] / notSpam
  
  newTup = (cp1, cp2, cp3, cp4)
  condP_dict.update({key: newTup})

100%|██████████| 3000/3000 [00:00<00:00, 271341.34it/s]


In [ ]:
from tqdm import tqdm

for key, value in tqdm(condP_dict.items()):
  #print(key, value)
  summ = 0
  for v in value:
    summ += v
  print(f"\n{summ}, {key}")

# Step 4. 

For each email, compute a probability score 

$$ P(\text{spam} | word_1, word_2, ... word_n )=  ~P(\text{spam})   \prod_{i}^{n}  P( word_i | \text{spam}) $$ 
and
$$ P(\lnot\text{spam} | word_1, word_2, ... word_n )=  ~P(\lnot\text{spam})   \prod_{i}^{n}  P( word_i |~\lnot\text{spam}) $$ 

where  $email = (word_1, word_2, ..., word_n)$

Then compare the two scores and classify as _Spam (1) or Not (0)_ based on whichever one is higher. 


In [ ]:
#Write your code here for P(spam | word_1 .. word_n) > P(~spam | word_1 .. word_n)
isSpam, notSpam, totalEmails, spamCount, hamCount = probSpam()
probScore_dict = {}

for key, value in tuple_dict.items(): # per email
  cp1Product = 1
  cp3Product = 1

  for i in range(len(value) - 1): # per word
    if value[i] == 1:
      conditionalTuple = condP_dict[words[i]] # grab word
      cp1 = conditionalTuple[0] # grab conditional probability of word given spam
      cp3 = conditionalTuple[2] # grab conditional probability of word given ham

      cp1Product *= cp1
      cp3Product *= cp3

  if cp1Product * isSpam > cp3Product * notSpam:
    probScore_dict.update({key: 1})
  else: 
    probScore_dict.update({key: 0})

# Step 5

Lastly, compute accuracy of your predictions. 

`Accuracy` = `sum(your_predictions == data['Prediction']) / len(data)`

In [ ]:
#Write your code here for accuracy score

probSum = 0

for key, value in tuple_dict.items():
  if probScore_dict[key] == value[-1]:
    probSum += 1

accuracy = probSum / len(data)
print(accuracy)

0.7140371229698376


<hr/>

1. Use `emails.csv` instead of `emails_binary.csv` in your experiments. Think about adjustments to be made to the joint and conditional probability formulas. `emails.csv` is upload on Moodle under PA.4. 


2. Include $\alpha$ in equation 2 for classification: 

$$ \alpha~𝑃~(\text{spam}| 𝑤𝑜𝑟𝑑1, 𝑤𝑜𝑟𝑑2,...𝑤𝑜𝑟𝑑𝑛)> \alpha~𝑃~( ¬ \text{spam}| 𝑤𝑜𝑟𝑑1, 𝑤𝑜𝑟𝑑2,... 𝑤𝑜𝑟𝑑𝑛) $$

3. Identify top 10 words most indicative of an email being Spam and top 10 words most indicative of an email being Ham. 

4. Use common-sense preprocessing steps to improve your accuracy scores. 

In [1]:
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/fahadsultan/csc343/main/programming_assignments/PA.4./emails.csv', index_col=0)
data.head()

,the,to,ect,and,for,of,a,you,hou,in,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
Email No.,,,,,,,,,,,,,,,,,,,,,
Email 1,0,0,1,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Email 2,8,13,24,6,6,2,102,1,27,18,...,0,0,0,0,0,0,0,1,0,0
Email 3,0,0,1,0,0,0,8,0,0,4,...,0,0,0,0,0,0,0,0,0,0
Email 4,0,5,22,0,5,1,51,2,10,1,...,0,0,0,0,0,0,0,0,0,0
Email 5,7,6,17,1,5,2,57,0,9,3,...,0,0,0,0,0,0,0,1,0,0


In [132]:
badwords = data[data['Prediction'] != data['myPreds']]
badWordCount = list(badwords)

badWordList = []
i = 0
while i < 50:
  i += 1
  word = max(badWordCount)
  badWordCount.remove(word)
  badWordList.append(word)

betterWordCounts = data.groupby('Prediction').sum()
removedWords = betterWordCounts.drop(badWordList, axis = 1)



totalWordSum = data[data.columns[:-1]].sum()
# totalWordSum = 6,026,877

pSpam = data['Prediction'].mean()
pHam = 1 - pSpam

word_counts = data.groupby('Prediction').sum()

# cp1 = word_counts.loc[1] / totalWordSum
# cp3 = word_counts.loc[0] / totalWordSum

cp1 = removedWords.loc[1] / totalWordSum
cp3 = removedWords.loc[0] / totalWordSum


def get_prediction(email):
  words_in_email = email[email == 1].index 
  scoreSpam = cp1.loc[words_in_email].prod() * pSpam
  scoreHam = cp3.loc[words_in_email].prod() * pHam
  alpha = 1 / (scoreSpam + scoreHam)

  my_prediction = 1 if (scoreSpam * alpha) > (scoreHam * alpha) else 0 #ternary
  return my_prediction

data['myPreds'] = data[data.columns[:-2]].apply(get_prediction, axis = 1)

accuracy = (data['Prediction'] == data['myPreds']).sum() / len(data)
#accuracy, totalWordSum.sum(), word_counts
accuracy


0.8312064965197216

In [125]:
badwords = data[data['Prediction'] != data['myPreds']]
badWordCount = list(badwords)

badWordList = []
i = 0
while i < 100:
  i += 1
  word = max(badWordCount)
  badWordCount.remove(word)
  badWordList.append(word)


betterWordCounts = data.groupby('Prediction').sum()
removedWords = betterWordCounts.drop(badWordList, axis = 1)
removedWords

,the,to,ect,and,for,of,a,hou,in,on,...,jay,valued,lay,infrastructure,military,allowing,ff,dry,myPreds,Accuracy
Prediction,,,,,,,,,,,,,,,,,,,,,
0,24506,21486,22885,9671,11508,6763,181711,9532,31953,37750,...,61,12,274,7,3,13,2475,27,0,3057.87007
1,9839,10519,3719,6236,4653,6824,105425,938,22871,18810,...,4,43,233,15,31,8,2256,9,635,1249.12993


<center>
<img width="50%" src="https://raw.githubusercontent.com/fahadsultan/csc343/main/programming_assignments/PA.4./Always-Has-Been.png">
</center>